The goal of this notebook is to prototype a web scraper for Penny Acade news posts. I want to assemble a dataset so that I can later train a language model based on it.

In [33]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime
import sqlite3
import time
import re
import random

We need to iterate through each news post. The structure of the URL is as follows:

`https://www.penny-arcade.com/news/post/2023/03/03/moisture-farm`

The first part of the URL is constant. The second part is the year, the third part is the month, and the fourth part is the day. 

If you input just the date (e.g. `https://www.penny-arcade.com/news/post/2023/03/03/`), it will provide the full URL for the post. We can use this fact and the datetime library to iterate through without having to manually determine each URL.

In [34]:
# start_date = datetime.date(1998, 11, 18)
# end_date = datetime.date(2023, 3, 3)

start_date = datetime.date(2005, 1, 1)
end_date = datetime.date(2023, 3, 3)

delta = datetime.timedelta(days=1)

dates = pd.DataFrame()
while start_date <= end_date:
    dates = pd.concat([dates, pd.DataFrame({'date': [start_date]})])
    start_date += delta

dates = dates.reset_index(drop=True)
dates.head()

,date
0,2004-01-01
1,2004-01-02
2,2004-01-03
3,2004-01-04
4,2004-01-05


In [35]:
# create a list of urls
# e.g. https://www.penny-arcade.com/news/post/2023/03/03/
urls = []
for row in range(len(dates)):
    date = dates['date'][row].strftime('%Y-%m-%d').split('-')

    year = date[0]
    month = date[1].zfill(2)
    day = date[2].zfill(2)
    urls.append(f'https://www.penny-arcade.com/news/post/{year}/{month}/{day}/')

In [36]:
# create the sqlite database
conn = sqlite3.connect('penny_arcade.db')
conn.execute('''CREATE TABLE IF NOT EXISTS posts
                (id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                title TEXT,
                text TEXT,
                author TEXT,
                tags TEXT,
                comic TEXT)''')
conn.close()

In [37]:
conn = sqlite3.connect('penny_arcade.db')

# test_urls = ['https://www.penny-arcade.com/news/post/1998/11/19/','https://www.penny-arcade.com/news/post/2023/03/03/']

# iterate over each url
for url in urls:
# for url in test_urls:
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')

    if soup.find('main', {'id': 'main', 'class': 'error-page'}):
        # print(f'{url} is not a valid date')
        continue
    
    # get the date from the url
    # date = url.split('/')[-2:]
    match = re.search(r'/(\d{4})/(\d{2})/(\d{2})/', url)
    date = match.group(1) + '-' + match.group(2) + '-' + match.group(3)

    # get the title
    # it will be an h1 that comes after <p class="details date">
    title = soup.find('p', {'class': 'details date'}).find_next('h1').text

    # get the text
    # look for <section class="post-text">
    # then look for <p> tags inside of that
    # there can be multiple <p> tags and multiple <section> tags
    # so we need to iterate over each <section> tag
    # and then iterate over each <p> tag
    text = ''
    for section in soup.find_all('section', {'class': 'post-text'}):
        for p in section.find_all('p'):
            text += p.text
            text += '\n'

    # get the author
    # look for <p class="details author">
    author = soup.find('p', {'class': 'details author'}).text
    author = author[3:] # remove the 'by ' from the beginning
    if author == 'Tycho':
        author = 'Tycho Brahe'
    elif author == 'Gabe':
        author = 'Johnathan Gabriel'

    # get the tags
    # look for <ul class="tags">
    # then look for <li> tags inside of that
    # there can be multiple <li> tags
    # so we need to iterate over each <li> tag
    tags = ''
    for li in soup.find('ul', {'class': 'tags'}).find_all('li'):
        tags += li.text
        tags += ', ' if li != soup.find('ul', {'class': 'tags'}).find_all('li')[-1] else ''

    # get the associated comic url
    # we can simply transform the url to get the comic url
    # e.g. https://www.penny-arcade.com/news/post/2023/03/03/
    # becomes https://www.penny-arcade.com/comic/2023/03/03/
    comic = url.replace('news/post', 'comic')   

    # print the data
    # print(f'date: {date}')
    # print(f'title: {title}')
    # print(f'text: {text}')
    # print(f'author: {author}')
    # print(f'tags: {tags}')
    # print(f'comic: {comic}')

    # insert the data into the database
    conn.execute('''INSERT INTO posts (date, title, text, author, tags, comic)
                    VALUES (?, ?, ?, ?, ?, ?)''', (date, title, text, author, tags, comic))
    conn.commit()

    # wait  a random amount of time between 0.5 and 3 seconds
    # helps avoid getting blocked by the website
    time.sleep(random.uniform(0.5, 2))

conn.close()

    

In [38]:
# show the data
conn = sqlite3.connect('penny_arcade.db')
table = pd.read_sql_query('SELECT * FROM posts', conn)

conn.close()
table.tail()

,id,date,title,text,author,tags,comic
475,476,2004-12-24,"The Last Christmas, Page Four",This season's presentation of The Last Christm...,Tycho Brahe,,https://www.penny-arcade.com/comic/2004/12/24/
476,477,2004-12-27,"The Last Christmas, Page Five",\nThe Last Christmas cycle is now complete. I...,Tycho Brahe,,https://www.penny-arcade.com/comic/2004/12/27/
477,478,2004-12-28,Brothers In Arms,\nWe had the opportunity to do something reall...,Johnathan Gabriel,,https://www.penny-arcade.com/comic/2004/12/28/
478,479,2004-12-29,"The Sony Syndrome, Final","\nGod dammit, we couldn't resist capping that ...",Tycho Brahe,,https://www.penny-arcade.com/comic/2004/12/29/
479,480,2004-12-31,A Being Of Indescribable Power,"\nIt was idle speculation when, a couple years...",Tycho Brahe,,https://www.penny-arcade.com/comic/2004/12/31/
